### Python database connect

In [11]:
import sqlite3
con = sqlite3.connect('c:/sqlite/testdb')

In [12]:
cur = con.cursor() #데이터 베이스 open

In [4]:
cur.execute('create table usertable2(id char, name char, email char, birth int)')
# 파이썬에서 테이블 생성

In [5]:
cur.execute("insert into usertable2 values('lee1', 'hw', 'naver', 1994)")
cur.execute("insert into usertable2 values('lee2', 'gw', 'gmail', 1993)")
cur.execute("insert into usertable2 values('lee3', 'ch', 'daum', 1993)")
cur.execute("insert into usertable2 values('lee4', 'gd', 'nate', 1991)")
# 테이블에 데이터 생성

In [6]:
con.commit() #커밋하기

In [7]:
con.close() #종료 데이터베이스 닫기

In [1]:
import sqlite3
con = sqlite3.connect('c:/sqlite/testdb')

In [2]:
cur = con.cursor()

In [3]:
cur.execute('select * from usertable2')

In [4]:
while True:
    row = cur.fetchone()
    if row == None:
        break;
    data1 = row[0]
    data2 = row[1]
    data3 = row[2]
    data4 = row[3]
    print(data1, '|', data2, '|', data3, '|', data4)

lee1 | hw | naver | 1994
lee2 | gw | gmail | 1993
lee3 | ch | daum | 1993
lee4 | gd | nate | 1991


####  테이블을 만들고 테이블에 내용 입력하기

In [ ]:
import sqlite3
con = sqlite3.connect('c:/sqlite/testdb')
cur = con.cursor()

In [5]:
cur.execute('create table producttable2(pcode char, pname char, price int, amount int)')

OperationalError: table producttable2 already exists

In [9]:
while True:
    data1 = input('제품 코드 =>: ')
    if data1 =='':
        break
    data2 = input('제품명    =>: ')
    data3 = input('가격      =>: ')
    data4 = input('재고수량  =>: ')
    cur.execute("insert into producttable2 values(:pcode, :pname, :price, :amount)", {'pcode':data1, 'pname':data2, 'price':int(data3), 'amount':int(data4)})
    con.commit()
    
    cur.execute("select * from producttable2")
    print('제품코드 제품명       가격 재고수량')
    print('-----------------------------------')
    
    while True:
        row = cur.fetchone()
        if row == None:
            break;
        data1 = row[0]
        data2 = row[1]
        data3 = row[2]
        data4 = row[3]
        print('%-5s    %-10s   %-4d   %-3d' %(data1, data2, data3, data4))
    con.close()

제품 코드 =>: p0004
제품명    =>: manualpad
가격      =>: 3000
재고수량  =>: 6


ProgrammingError: Cannot operate on a closed database.

#### 크롤러 만들기  
#### 스크리핑 데이터 저장하기

In [28]:
import requests
import lxml.html
import re
import time
import sqlite3

con = sqlite3.connect('c:/sqlite/testdb')
cur = con.cursor()
cur.execute('drop table if exists ebook')
cur.execute('create table ebook(url char, title char, price char, content char)')
#데이터 베이스 연결 후 테이블 생성

session = requests.session() # 스크래핑할 사이트 선택
res = requests.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
root = lxml.html.fromstring(res.content)
root.make_links_absolute(res.url) #절대 경로 설정하기

for a in root.cssselect('.book_tit a'):
    url = a.get('href')
    print(url)
    time.sleep(2)
    res = session.get(url)
    root = lxml.html.fromstring(res.content)
    title = root.cssselect('.store_product_info_box h3')[0].text
    print('제목: ', title)
    price = root.cssselect('.pbr strong')[0].text
    print('가격: ', price)
    print('목차:')
    content = ''
    for p in root.cssselect('#tabs_3 .detail_conbox p'):
        content = p.text
        if content != None:
            content = re.sub('\s+', '', content)
        if content == None or content == '':
            continue
        print(p.text)
        content += p.text +'\n'
    print(content)
    cur.execute('insert into ebook values(:url, :title, :price, :content)', {'url':url, 'title':title, 'price':price, 'content':content})
    # 스크래핑한 데이터를 테이블에 저장한다.
con.commit()
con.close()
    

http://www.hanbit.co.kr/store/books/look.php?p_code=B2098898028
제목:  나도 글 좀 잘 쓰면 소원이 없겠네
가격:  13,320
목차:
프롤로그 다시, 글쓰기에 앞서
인트로 당신의 어릴 적 사진 같은 글을 찍어 봅시다
이 책의 핵심 구성
들어가기 전에, 잠깐 점검
01 내가 좋아하는 낱말 20개 이상 쓰기
02 내가 좋아하는 속담이나 격언 10문장 이상 쓰기
03 끝말잇기
04 삼행시 쓰기
05 브레인스토밍 글쓰기
1주차 일기 쓰기 / 어라, 또 일기라고?
1일차 보고 들은 것 쓰기
2일차 감정을 문장으로 묘사하기
3일차 경험한 사건을 문장으로 묘사하기
4일차 보고 듣고 생각한 것 쓰기
5일차 나의 하루
1일차 경험을 정리하기 
2일차 경험을 짧은 글로 풀어 보기
3일차 기행문 쓰기
4일차 경험에서 생각으로 확장하는 글쓰기
5일차 나의 경험과 생각 쓰기
1일차 무엇이든 보고 나면 무조건 메모하기
2일차 키워드를 문장으로 정리하기
3일차 좋으나 싫으나 팩트로 설명하기
4일차 논리적으로 설득하기
5일차 좋아하는 작품으로 리뷰 쓰기
1일차 무엇을 쓸까?
2일차 무슨 이야기를 어떻게 전개할까?
3일차 좀 더 쉽고 재미있게 쓰자
4일차 퇴고를 합시다!
5일차 나만의 글쓰기
01 내 자서전의 제목 쓰고 고르기
02 내 주변 사람들을 한 단어로 묘사하기
03 드라마나 웹툰 리뷰 쓰기
04 캐릭터나 스토리를 바꾸는 글쓰기
05 일상의 대화를 옮겨 적고 느낀 점 쓰기
06 기자가 되어 이슈가 된 인터넷 기사 바꿔 쓰기
07 오감이 드러나는 글쓰기
08 새로 방문한 곳에 대해 묘사하는 글쓰기
09 마음에 드는 글 필사하기
10 관점을 바꾼 일기 쓰기
11 나를 화나게 한 사건과 웃게 한 사건에 관해 기술하기
12 가장 인상적인 여행지를 골라 기행문 쓰기
13 자기 자신의 장점을 어필하는 자기 소개서 쓰기
14 내 생각과 말을 관찰하여 심리 상태 묘사하기
15 지난 4주간의 글쓰기 훈련을 마친 소감 쓰기
작가 추천 도서 목록
에

제목:  IT CookBook, 리눅스 실습 for Beginner
가격:  24,000
목차:
01. 리눅스의 개요
02. 우분투 리눅스 배포판
03. 가상머신의 개요
04. VMware 설치
05. 가상머신 생성
06. VMware의 특징과 실습 준비
요약/연습문제
01. 우분투 데스크톱 설치
02. 우분투 서버 설치
03. 쿠분투 설치
요약/연습문제
01. 리눅스의 기초
02. 자동 완성 기능
03. vi 에디터와 도움말
04. 리눅스 마운트
요약/연습문제
01. 리눅스 기본 명령어
02. 네트워크 명령어
03. 파일 압축과 묶기
04. 시스템 설정
요약/연습문제
01. 사용자 관리
02. 파일의 소유와 허가권
03. 프로세스, 서비스(데몬)
요약/연습문제
01. 프로그램 설치 명령어 dpkg
02. 패키지 설치 명령어 apt-get
03. 응급 복구와 GRUB 부트로더
요약/연습문제
01. GNOME 데스크톱 환경 설정
02. X 윈도우 응용 프로그램의 종류
03. GNOME 데스크톱 설치
요약/연습문제
01. 우분투 소프트웨어 센터
02. KDE 데스크톱 설치와 사용
03. 리눅스에 윈도우 설치
요약/연습문제
01. 디스크와 파티션
02. 디스크 추가
03. 사용자별 공간 할당
요약/연습문제
01. RAID의 개념과 RAID 레벨
02. RAID 구축을 위한 준비
03. RAID의 다양한 레벨 구축
요약/연습문제
01. 셸의 개념과 특징
02. 셸 스크립트 프로그래밍 기본
03. 셸 스크립트 프로그래밍 응용
요약/연습문제
01. 텔넷 서버
02. SSH 서버
03. VNC 서버
요약/연습문제
01. APM의 개요와 설치
02. XE를 활용한 웹 사이트 구축
03. FTP 서버 설치와 운영
요약/연습문제
01. 미니 프로젝트의 개요
02. 실무 환경의 우분투 설치
03. RAID 6 구축
04. 백업 자동화
05. Visual Studio Code 개발 환경 구축
요약/연습문제
찾아보기
찾아보기찾아보기

http://www.hanbit.co.kr/sto

제목:  NO BULLSHIT 수학&물리 가이드
가격:  28,000
목차:
1.1 방정식 풀이
1.2 수
1.3 수 표현
1.4 변수
1.5 함수와 역함수
1.6 대수학의 기본 법칙
1.7 2차 방정식 풀이
1.8 지수
1.9 로그
1.10 데카르트 평면
1.11 함수
1.12 함수 모음
1.13 함수 변환
1.14 기하학
1.15 삼각법
1.16 삼각 함수 공식
1.17 원
1.18 타원
1.19 쌍곡선
1.20 선형 연립 방정식 풀이
1.21 복리
1.22 집합 표기
1장 연습문제
2.1 서론
2.2 운동학
2.3 미적분학 기초
2.4 미적분학과 운동학
2장 연습문제
3.1 대자연
3.2 벡터
3.3 기저
3.4 벡터곱
3.5 복소수
3장 연습문제
4.1 서론
4.2 포물체 운동
4.3 힘
4.4 힘 다이어그램
4.5 운동량
4.6 에너지
4.7 등속 원운동
4.8 각운동
4.9 단순 조화 운동
4.10 결론
4장 연습문제
5.1 서론
5.2 개요
5.3 무한
5.4 극한
5.5 극한 공식
5.6 미분
5.7 미분 공식
5.8 미분법
5.9 고계 미분
5.10 최적화 알고리즘
5.11 음함수 미분법
5.12 적분
5.13 리만 합
5.14 미적분학의 기본 정리
5.15 적분법
5.16 적분의 응용
5.17 이상 적분
5.18 수열
5.19 급수
5.20 결론
5장 연습문제
부록 A 표기
부록 B 상수, 단위, 변환 비율
부록 C SymPy 지도서
부록 D 공식
부록D공식부록 D 공식

http://www.hanbit.co.kr/store/books/look.php?p_code=B7019463330
제목:  라임튜브와 함께하는 우리 아이 유튜브 스타 만들기
가격:  13,500
목차:
먼저 크리에이터를 알아보자! 
그럼 키즈 크리에이터는 뭘까? 
평범했던 라임이네 가족
유튜브가 눈에 보이다! 
끼가 많던 라임이 
라임파파가 운영하던 장난감 채널
라임이가 등장하자 조회수가 2배 증가했어요! 
유튜브 키즈 데이 행사를 통해 한 단계 성장하다! 
라임이가 나온 

### 크롤링, 데이터베이스 저장 모듈화 하기

###### 1 테이블 생성 함수 만들기

In [30]:
def table_make(name):
    con = sqlite3.connect('c:/sqlite/testdb')
    cur = con.cursor()
    cur.execute('drop table if exists name')
    cur.execute('create table name(url char, title char, price char, content char)')
    #데이터 베이스 연결 후 테이블 생성

##### 2 크롤러 생성

In [ ]:
def crawler():
    session = requests.session() # 스크래핑할 사이트 선택
    res = requests.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    root = lxml.html.fromstring(res.content)
    root.make_links_absolute(res.url) #절대 경로 설정하기

    for a in root.cssselect('.book_tit a'):
        url = a.get('href')
        print(url)
        time.sleep(2)
        res = session.get(url)
        root = lxml.html.fromstring(res.content)
        title = root.cssselect('.store_product_info_box h3')[0].text
        print('제목: ', title)
        price = root.cssselect('.pbr strong')[0].text
        print('가격: ', price)
        print('목차:')
        content = ''
        for p in root.cssselect('#tabs_3 .detail_conbox p'):
            content = p.text
            if content != None:
                content = re.sub('\s+', '', content)
            if content == None or content == '':
                continue
            print(p.text)
            content += p.text +'\n'
        print(content)
        content += p.text +'\n'

##### 3 데이터 저장 함수

In [ ]:
def save_data():
    print(content)
    cur.execute('insert into ebook values(:url, :title, :price, :content)', {'url':url, 'title':title, 'price':price, 'content':content})
    # 스크래핑한 데이터를 테이블에 저장한다.
    con.commit()
    con.close()

======================================================================================================

In [83]:
import requests
import lxml.html
import re
import time
import sqlite3

session = requests.session() # 스크래핑할 사이트 선택
res = requests.get('https://comic.naver.com/index.nhn')
root = lxml.html.fromstring(res.content)
root.make_links_absolute(res.url) #절대 경로 설정하기

for a in root.cssselect('.genreRecomImg2 a'):
    url = a.get('href')
    print(url)

    res = session.get(url)
    root = lxml.html.fromstring(res.content)
    
    title = root.cssselect('.detail h2')[0].text
    title = re.sub('\s+', '', title)
    print('제목: ', title)
    
    writer = root.cssselect('.wrt_nm')[0].text
    writer = re.sub('\s+', '', writer)
    print('작가: ', writer)
    
    content = root.cssselect('.detail p')[0].text
    print('내용: ', content)
    
    genre = root.cssselect('.genre')[0].text
    print('장르: ', genre)
    
    age = root.cssselect('.age')[0].text
    print('연령: ', age)
    
    '''if content == None or content == '':
            continue
        print(p.text)
        content += p.text +'\n'
    print(content)
    cur.execute('insert into ebook values(:url, :title, :price, :content)', {'url':url, 'title':title, 'price':price, 'content':content})
    # 스크래핑한 데이터를 테이블에 저장한다.'''

https://comic.naver.com/webtoon/list.nhn?titleId=732071
제목:  가령의정체불명이야기
작가:  가령
내용:  어느 날... 인어가 내게 말을 걸어왔다. 
장르:  옴니버스, 일상, 판타지, 드라마
연령:  15세 이용가
https://comic.naver.com/webtoon/list.nhn?titleId=21815
제목:  히어로메이커
작가:  빤쓰
내용:  왕은 영웅이 되고 싶어하는 공주의 소원을 들어주기로 전격 결심! 공주를 속이고 마치 영웅이 된 것처럼 만들기 위해 온 나라가 연극을 하게 되는데..파란만장한 그들만의 눈물겨운 영웅만들기의 대장정이 펼쳐집니다~
장르:  옴니버스, 판타지
연령:  전체연령가
https://comic.naver.com/webtoon/list.nhn?titleId=694946
제목:  귀전구담
작가:  QTT
내용:  사람들은 모여 무서운 이야기라며 귀신이야기를 한다.
장르:  옴니버스, 판타지, 드라마, 감성, 스릴러
연령:  12세 이용가
https://comic.naver.com/webtoon/list.nhn?titleId=22897
제목:  호랭총각
작가:  강호진
내용:  매일 아침 씨리얼 한 사발로 하루 끼니를 때우는 가난한 나무꾼 호랭총각.배고프지 않고, 외롭지 않은 삶을 꿈꾸는 선량한 호랭총각과 토마호크 대신 연못에서 솟아난 덩치큰 친구와 함께하는 이상한 이야기.
장르:  옴니버스, 개그, 드라마
연령:  12세 이용가
https://comic.naver.com/webtoon/list.nhn?titleId=695796
제목:  내일
작가:  라마
내용:  다 되는데 취업만은 안되는 취준생 최준웅(27세,남)
장르:  옴니버스, 일상, 판타지, 드라마
연령:  전체연령가
https://comic.naver.com/webtoon/list.nhn?titleId=729037
제목:  호러전파상
작가:  봄소희/김선희
내용:  "명심하세요. 전자기기로 인한 피해는 이 가게에서 책